## Sampling 3DEP Example

### Purpose
Demonstrate how to sample the 3DEP 1m DEMs at GEDI L4A footprints

In [ ]:
import time
import matplotlib.pyplot as plt
from sliderule import gedi, earthdata, raster
import sliderule

### Initialize client

In [ ]:
gedi.init("slideruleearth.io", verbose=True)

### Specify region of interest from geojson

In [ ]:
poly_fn = 'grandmesa.geojson'
region = sliderule.toregion(poly_fn)

### Query USGS "The National Map" API for 3DEP 1m products in area of interest

In [ ]:
geojson = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', polygon=region["poly"])

### Make GEDI L4A subset request with 3DEP sampling

In [ ]:
# Build GEDI L4A Request Parameters
parms = {
    "poly": region["poly"],
    "degrade_flag": 0,
    "l2_quality_flag": 1,
    "beam": 0,
    "samples": {"3dep": {"asset": "usgs3dep-1meter-dem", "catalog": geojson}} 
}

# Latch Start Time
perf_start = time.perf_counter()

# Request GEDI Data
gedi04a = gedi.gedi04ap(parms, resources=['GEDI04_A_2019123154305_O02202_03_T00174_02_002_02_V002.h5'])
                        
# Latch Stop Time
perf_stop = time.perf_counter()

# Display Statistics
perf_duration = perf_stop - perf_start
print("Completed in {:.3f} seconds of wall-clock time".format(perf_duration))
print("Received {} footprints".format(gedi04a.shape[0]))
if len(gedi04a) > 0:
    print("Beams: {}".format(gedi04a["beam"].unique()))

In [ ]:
gedi04a

### Massage DataFrame: trim NaN and no-data rows and flatten samples

In [ ]:
gdf = gedi04a[gedi04a["3dep.value"].notna()]

In [ ]:
def getFirstValue(x):
    if type(x["3dep.value"]) == float:
        return x['3dep.value']
    else:
        return x['3dep.value'][0]
gdf["3dep"] = gdf.apply(lambda x: getFirstValue(x), axis=1)

In [ ]:
gdf = gdf[gdf["3dep"] > -9999.0]

### Plot elevations using coordinates

In [ ]:
f, ax = plt.subplots(1, 2, figsize=[12,8])
ax[0].set_title("GEDI")
ax[0].set_aspect('equal')
gdf.plot(ax=ax[0], column='elevation', cmap='inferno', s=0.1)
ax[1].set_title("3DEP")
ax[1].set_aspect('equal')
gdf.plot(ax=ax[1], column='3dep', cmap='inferno', s=0.1)

### Plot comparison of elevations

In [ ]:
fig,ax = plt.subplots(num=None, figsize=(10, 8))
fig.set_facecolor('white')
fig.canvas.header_visible = False
ax.set_title("Elevations between GEDI and 3DEP")
ax.set_xlabel('UTC')
ax.set_ylabel('height (m)')
ax.yaxis.grid(True)
sc1 = ax.scatter(gdf.index.values, gdf["elevation"].values, c='blue', s=2.5)
sc2 = ax.scatter(gdf.index.values, gdf["3dep"].values, c='green', s=2.5)
plt.show()

### Plot Histogram of Differences

In [ ]:
gdf['elev_diff'] = gdf['elevation'] - gdf['3dep']

In [ ]:
plt.hist(gdf['elev_diff'], bins=128)